# Exécution optimale d'une série d'ordres

On se donne un welfare (ex : utilitaire, min, max, Nash, etc.) définit sur $\cup_n\mathbb{R}^n$ à valeurs dans $\mathbb{R}$.
On peut montrer que, s'il existe un ordre total $\preceq$ (pour ce welfare) sur l'ensemble des limit orders possibles, alors si $o_a$ est un ordre ask à un prix $p_a$ et $o_b$ est un ordre bid à un prix $p_b$, alors si $p_a < p_b$, on a $o_b \preceq o_a$. Pour montrer que $\preceq$ n'existe pas pour ce welfare donné, il suffit donc d'exhiber une séquence d'ordres $(o_1, ..., o_m)$ telle qu'il existe $1\leq i\neq j\leq m$ avec $o_i$ un ordre ask, $o_j$ un ordre bid avec $p_i < p_j$ et pour toutes les permutations $(o_{\sigma(1)}, ...o_{\sigma(m)})$ maximisant le welfare $w(o_{\sigma(1)}, ..., o_{\sigma(m)})$ sont telles que $\sigma^{-1}(i) < \sigma^{-1}(j)$ : dans toutes les permutations de la séquence qui maximisent le welfare, l'ordre ask $o_i$ est avant l'ordre bid $o_j$.

In [10]:
from atom import *
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import itertools as it
plt.rcParams['figure.figsize'] = (15,10)

class AutomatonNoPop(Trader):
    def __init__(self, market, initial_assets=None, cash=0, orders_list=[]):
        '''order_list est une liste de la forme (asset,direction,price,qty)'''
        Trader.__init__(self, market, initial_assets, cash)
        self.orders_list = orders_list
    def __str__(self):
        return "Automaton "+ super().__str__()
    def decide_order(self, market, asset):
        if self.orders_list == []:
            return None
        else:
            a, d, p, q = self.orders_list[0]
            return LimitOrder(a, self, d, p, q)

class Automaton(Trader):
    def __init__(self, market, initial_assets=None, cash=0, orders_list=[]):
        '''order_list est une liste de la forme (asset,direction,price,qty)'''
        Trader.__init__(self, market, initial_assets, cash)
        self.orders_list = orders_list
    def __str__(self):
        return "Automaton "+ super().__str__()
    def decide_order(self, market, asset):
        if self.orders_list == []:
            return None
        else:
            a, d, p, q = self.orders_list.pop()
            return LimitOrder(a, self, d, p, q)

def apply_perm_to_lst(l1, l2):
    # l1 = permutation de [[1,n]] avec n = len(l2)
    return [l2[l1[i]-1] for i in range(len(l2))]

#print(apply_perm_to_lst([3,2,1], ['a','b','c']))

## Calcul des meilleures séquences

### Séquences ne respectant pas $\preceq$ pour $W_\min$ et $W_N$

In [12]:
lst_best = []
best_wealth = 0
perm_list = it.permutations([1,2,3])

# ATTENTION :
# Ne pas définir les traders avant la boucle puis faire une perm des traders,
# sinon à la fin de la première étape de la boucle, même si le marché est réinitialisé, l'agent ne le sera pas
# et son cash initial à l'étape 2 sera son cash final de l'étape 1...

p1 = 1463; p2 = 1248; p3 = 5528
q1 = 3; q2 = 4; q3 = 6
c1 = 20932; c2 = 45856; c3 = 12339
n1 = 4; n2 = 24; n3 = 5

for p in perm_list:
    file = open('trace'+str(p)+'.dat', 'w')
    m = Market(['asset'], out=file, trace='all')
    t1 = Automaton(m, [n1], c1, [('asset', 'ASK', p1, q1)])
    t2 = Automaton(m, [n2], c2, [('asset', 'ASK', p2, q2)])
    t3 = Automaton(m, [n3], c3, [('asset', 'BID', p3, q3)])
    p_t = apply_perm_to_lst(p, [t1,t2,t3])
    for t in p_t:
        m.add_trader(t)
    m.run_once(shuffle=False)
    # C'est ici qu'on définit le welfare !
    # min ou np.prod
    w = min([t.get_wealth(m) for t in m.traders])
    print(p, [t.get_wealth(m) for t in m.traders], w)
    if w > best_wealth:
        lst_best = [p]
        best_wealth = w
    elif w == best_wealth:
        lst_best.append(p)
    file.close()
print("\n", lst_best)

(1, 2, 3) [26784, 80108, 20514] 20514
(1, 3, 2) [30849, 52174, 178528] 30849
(2, 1, 3) [80108, 26784, 20514] 20514
(2, 3, 1) [161408, 57099, 43044] 43044
(3, 1, 2) [39979, 43044, 178528] 39979
(3, 2, 1) [39979, 178528, 43044] 39979

 [(2, 3, 1)]


### Séquences ne respectant pas $\preceq$ pour $W_\max$

In [13]:
lst_best = []
best_wealth = 0
perm_list = it.permutations([1,2,3])

# ATTENTION :
# Ne pas définir les traders avant la boucle puis faire une perm des traders,
# sinon à la fin de la première étape de la boucle, même si le marché est réinitialisé, l'agent ne le sera pas
# et son cash initial à l'étape 2 sera son cash final de l'étape 1...

p1 = 1166; p2 = 1002; p3 = 2048
q1 = 1; q2 = 13; q3 = 11
c1 = 23500; c2 = 14969; c3 = 32763
n1 = 11; n2 = 15; n3 = 24

for p in perm_list:
    file = open('trace'+str(p)+'.dat', 'w')
    m = Market(['asset'], out=file, trace='all')
    t1 = AutomatonNoPop(m, [n1], c1, [('asset', 'ASK', p1, q1)])
    t2 = AutomatonNoPop(m, [n2], c2, [('asset', 'ASK', p2, q2)])
    t3 = AutomatonNoPop(m, [n3], c3, [('asset', 'BID', p3, q3)])
    p_t = apply_perm_to_lst(p, [t1,t2,t3])
    for t in p_t:
        m.add_trader(t)
    m.run_once(shuffle=False)
    w = max([t.get_wealth(m) for t in m.traders])
    print(p, [t.get_wealth(m) for t in m.traders], w)
    if w > best_wealth:
        lst_best = [p]
        best_wealth = w
    elif w == best_wealth:
        lst_best.append(p)
    file.close()
print("\n", lst_best)

(1, 2, 3) [34522, 29999, 56811] 56811
(1, 3, 2) [45146, 82797, 45689] 82797
(2, 1, 3) [29999, 34522, 56811] 56811
(2, 3, 1) [29999, 56811, 34522] 56811
(3, 1, 2) [81915, 46028, 45689] 81915
(3, 2, 1) [81915, 45689, 46028] 81915

 [(1, 3, 2)]


### Séquences ne respectant pas $\preceq$ pour $W_u$

In [16]:
lst_best = []
best_wealth = 0
perm_list = it.permutations([1,2,3,4])

# ATTENTION :
# Ne pas définir les traders avant la boucle puis faire une perm des traders,
# sinon à la fin de la première étape de la boucle, même si le marché est réinitialisé, l'agent ne le sera pas
# et son cash initial à l'étape 2 sera son cash final de l'étape 1...

p1 =  5060  ; q1 =  13  ; c1 =  30613  ; n1 =  27 
p2 =  6498  ; q2 =  5  ; c2 =  11807  ; n2 =  17 
p3 =  8103  ; q3 =  5  ; c3 =  19314  ; n3 =  17 
p4 =  7312  ; q4 =  1  ; c4 =  46352  ; n4 =  20

for p in perm_list:
    file = open('trace'+str(p)+'.dat', 'w')
    m = Market(['asset'], out=file, trace='all')
    t1 = AutomatonNoPop(m, [n1], c1, [('asset', 'ASK', p1, q1)])
    t2 = AutomatonNoPop(m, [n2], c2, [('asset', 'ASK', p2, q2)])
    t3 = AutomatonNoPop(m, [n3], c3, [('asset', 'BID', p3, q3)])
    t4 = AutomatonNoPop(m, [n4], c4, [('asset', 'BID', p4, q4)])
    p_t = apply_perm_to_lst(p, [t1,t2,t3,t4])
    for t in p_t:
        m.add_trader(t)
    m.run_once(shuffle=False)
    w = sum([t.get_wealth(m) for t in m.traders])
    print(p, [t.get_wealth(m) for t in m.traders], w)
    if w > best_wealth:
        lst_best = [p]
        best_wealth = w
    elif w == best_wealth:
        lst_best.append(p)
    file.close()
print("\n", lst_best)

(1, 2, 3, 4) [167233, 97827, 105334, 147552] 517946
(1, 2, 4, 3) [167233, 97827, 147552, 105334] 517946
(1, 3, 2, 4) [167233, 105334, 97827, 147552] 517946
(1, 3, 4, 2) [167233, 105334, 147552, 97827] 517946
(1, 4, 2, 3) [167233, 147552, 97827, 105334] 517946
(1, 4, 3, 2) [167233, 147552, 105334, 97827] 517946
(2, 1, 3, 4) [97827, 167233, 105334, 147552] 517946
(2, 1, 4, 3) [97827, 167233, 147552, 105334] 517946
(2, 3, 1, 4) [105017, 98144, 167233, 147552] 517946
(2, 3, 4, 1) [132041, 147688, 192592, 228037] 700358
(2, 4, 1, 3) [99265, 146114, 167233, 105334] 517946
(2, 4, 3, 1) [141533, 210017, 163485, 249394] 764429
(3, 1, 2, 4) [90119, 182448, 97827, 147552] 517946
(3, 1, 4, 2) [90119, 182448, 147552, 97827] 517946
(3, 2, 1, 4) [90119, 113042, 167233, 147552] 517946
(3, 2, 4, 1) [139663, 140066, 192592, 228037] 700358
(3, 4, 1, 2) [139663, 192592, 231992, 136111] 700358
(3, 4, 2, 1) [139663, 192592, 140066, 228037] 700358
(4, 1, 2, 3) [145300, 169485, 97827, 105334] 517946
(4, 1, 3,

### Séquences ne respectant pas $\preceq$ pour $W_\text{Lm}$

In [17]:
def leximin(x, y):
    n = len(x)
    x_cp = x[:]; y_cp = y[:]
    x_cp.sort()
    y_cp.sort()
    k = 0
    while k < n and x_cp[k] == y_cp[k]:
        k += 1
    if k == n:
        return [x, y]
    elif x_cp[k] < y_cp[k]:
        return [y]
    else:
        return [x]

# leximin([0,1],[1,0])
# leximin([.5,1],[1.5,0])

def leximin_lst(l):
    lst_max = [l[0][:]]
    max = l[0][:]
    for i in range(1,len(l)):
        if leximin(max, l[i][:]) == [l[i][:]]:
            lst_max = [l[i][:]]
            max = l[i][:]
        elif leximin(max, l[i][:]) == [max, l[i][:]]:
            lst_max.append(l[i][:])
    return lst_max

# l1 = [.5,1] ; l2 = [1,.7] ; l3 = [1.5,0] ; l4 = [.7,1]
# leximin_lst([l1,l2,l3,l4])

In [19]:
perm_list = it.permutations([1,2,3])

# ATTENTION :
# Ne pas définir les traders avant la boucle puis faire une perm des traders,
# sinon à la fin de la première étape de la boucle, même si le marché est réinitialisé, 
# l'agent ne le sera pas et son cash initial à l'étape 2 sera son cash final de l'étape 1...

p1 = 1463; p2 = 1248; p3 = 5528
q1 = 3; q2 = 4; q3 = 6
c1 = 20932; c2 = 45856; c3 = 12339
n1 = 4; n2 = 24; n3 = 5
welfares = dict()

for p in perm_list:
    file = open('trace'+str(p)+'.dat', 'w')
    m = Market(['asset'], out=file, trace='all')
    t1 = AutomatonNoPop(m, [n1], c1, [('asset', 'ASK', p1, q1)])
    t2 = AutomatonNoPop(m, [n2], c2, [('asset', 'ASK', p2, q2)])
    t3 = AutomatonNoPop(m, [n3], c3, [('asset', 'BID', p3, q3)])
    p_t = apply_perm_to_lst(p, [t1,t2,t3])
    for t in p_t:
        m.add_trader(t)
    m.run_once(shuffle=False)
    welfares[p] = [t.get_wealth(m) for t in m.traders]
    print(p, welfares[p])
    file.close()
lst_max = leximin_lst(list(welfares.values()))
print(lst_max)
# print([p for p in perm_list if welfares[p] in lst_max])

(1, 2, 3) [26784, 80108, 20514]
(1, 3, 2) [30849, 52174, 178528]
(2, 1, 3) [80108, 26784, 20514]
(2, 3, 1) [161408, 57099, 43044]
(3, 1, 2) [39979, 43044, 178528]
(3, 2, 1) [39979, 178528, 43044]
[[161408, 57099, 43044]]


## Trouver des séquences ne respectant pas $\preceq$

In [24]:
file = open('/dev/null', 'w')

for nb in range(100000):
    p3 = rd.randint(1500,9999)
    p4 = rd.randint(1500,9999)
    #p1 = rd.randint(1000, p3-1)
    #p2 = rd.randint(1000, p3-1)
    p1 = rd.randint(1000, min(p3, p4)-1)
    p2 = rd.randint(1000, min(p3, p4)-1)
    n1 = rd.randint(1,30)
    n2 = rd.randint(1,30)
    n3 = rd.randint(0,30)
    n4 = rd.randint(0,30)
    q1 = rd.randint(1,n1)
    q2 = rd.randint(1,n2)
    q3 = rd.randint(1,q1+q2)
    q4 = rd.randint(1,q1+q2)
    c1 = rd.randint(10000,50000)
    c2 = rd.randint(10000,50000)
    c3 = rd.randint(10000,50000)
    c4 = rd.randint(10000,50000)
    # Calcul
    lst_best = []
    best_wealth = 0
    #perm_list = it.permutations([1,2,3])
    perm_list = it.permutations([1,2,3,4])
    for p in perm_list:
        m = Market(['asset'], out=file)
        t1 = AutomatonNoPop(m, [n1], c1, [('asset', 'ASK', p1, q1)])
        t2 = AutomatonNoPop(m, [n2], c2, [('asset', 'ASK', p2, q2)])
        t3 = AutomatonNoPop(m, [n3], c3, [('asset', 'BID', p3, q3)])
        t4 = AutomatonNoPop(m, [n4], c4, [('asset', 'BID', p4, q4)])
        #p_t = apply_perm_to_lst(p, [t1,t2,t3])
        p_t = apply_perm_to_lst(p, [t1,t2,t3,t4])
        for t in p_t:
            m.add_trader(t)
        m.run_once(shuffle=False)
        # Welfare
        # C'est ici qu'il faut choisir quelle fonction d'utilité on prend !
        w = sum([t.get_wealth(m) for t in m.traders])
        if w > best_wealth:
            lst_best = [p]
            best_wealth = w
        elif w == best_wealth:
            lst_best.append(p)
    i = 0
    n = len(lst_best)
    #while i < n and lst_best[i][0] != 3:
    while i < n and (lst_best[i][0], lst_best[i][1]) not in [(3,4),(4,3)]:
        i += 1
    if i == n:
        print(lst_best)
        #print("p1 = ", p1, " ; q1 = ", q1, " ; c1 = ", c1, " ; n1 = ", n1, "\np2 = ", p2, " ; q2 = ", q2, " ; c2 = ", c2, " ; n2 = ", n2, "\np3 = ", p3, " ; q3 = ", q3, " ; c3 = ", c3, " ; n3 = ", n3)
        print("p1 = ", p1, " ; q1 = ", q1, " ; c1 = ", c1, " ; n1 = ", n1, "\np2 = ", p2, " ; q2 = ", q2, " ; c2 = ", c2, " ; n2 = ", n2, "\np3 = ", p3, " ; q3 = ", q3, " ; c3 = ", c3, " ; n3 = ", n3, "\np4 = ", p4, " ; q4 = ", q4, " ; c4 = ", c4, " ; n4 = ", n4)
        break

file.close()

[(1, 3, 4, 2), (2, 3, 4, 1), (3, 1, 4, 2), (3, 2, 4, 1)]
p1 =  1696  ; q1 =  8  ; c1 =  33734  ; n1 =  13 
p2 =  1569  ; q2 =  6  ; c2 =  13539  ; n2 =  13 
p3 =  2624  ; q3 =  14  ; c3 =  24887  ; n3 =  19 
p4 =  8321  ; q4 =  10  ; c4 =  21879  ; n4 =  8
